### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 2, 4)

In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-02-04
Yesterday: 2025-02-03


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-02-03'
ORDER BY a.name



(0, 1)

In [28]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % today
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-02-04'
ORDER BY a.name



(20, 1)

In [35]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-02-03' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [37]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

RecursionError: maximum recursion depth exceeded

In [ ]:
df.shape

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [ ]:
df[cols].style.format(format_dict)

In [ ]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [ ]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

In [ ]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [ ]:
sales = df[cols]
sales.shape

In [ ]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

In [ ]:
prices = pd.read_csv(input_file)
prices.shape

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-01-22,2025-02-03,5.15,5.35,110765210,5.35,5.10,3.88,B,5.35,0.10,18233803,2025-02-04
1,AIMIRT,2025-01-31,2025-02-03,10.50,10.50,366820,10.50,10.40,0.00,B,10.50,0.00,381807,2025-02-04
2,CPNREIT,2025-01-31,2025-02-03,12.10,12.20,2212100,12.40,12.10,0.83,B,12.20,0.10,2691021,2025-02-04
3,DIF,2025-01-30,2025-02-03,8.40,8.40,35916330,8.55,8.35,0.00,B,8.40,0.00,8428619,2025-02-04
4,GVREIT,2025-02-03,2025-02-03,6.75,6.60,1854989,6.75,6.50,-2.22,B,6.60,-0.15,894103,2025-02-04
5,JMART,2025-01-30,2025-02-03,11.30,10.80,18272786,12.00,10.80,-4.42,B,10.80,-0.10,4487067,2025-02-04
6,KCE,2025-01-29,2025-02-03,22.50,21.00,29023548,23.20,21.00,-6.67,B,21.00,-1.00,11916173,2025-02-04
7,MCS,2025-01-29,2025-02-03,6.25,6.10,538102,6.30,6.10,-2.40,B,6.10,-0.10,299278,2025-02-04
8,NER,2025-01-29,2025-02-03,4.98,4.88,27786086,5.05,4.88,-2.01,B,4.88,-0.02,12166889,2025-02-04
9,SENA,2025-01-29,2025-02-03,2.18,2.16,2578480,2.20,2.16,-0.92,B,2.16,-0.02,611103,2025-02-04


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-01-22 2025-02-04   5.15   5.35   3.88% 110,765,210   5.35   5.10 B Update
AIMIRT   2025-01-31 2025-02-04  10.50  10.50    0.0%     366,820  10.50  10.40 B Update
AWC      2025-01-29 2025-02-04   3.18   3.14  -1.26% 161,777,036   3.24   3.08 S Update
BCH      2025-01-30 2025-02-04  14.30  14.30    0.0%  17,985,510  14.60  14.20 S Update
CPNREIT  2025-01-31 2025-02-04  12.10  12.20   0.83%   2,212,100  12.40  12.10 B Update
DIF      2025-01-30 2025-02-04   8.40   8.40    0.0%  35,916,330   8.55   8.35 B Update
GVREIT   2025-02-03 2025-02-04   6.75   6.60  -2.22%   1,854,989   6.75   6.50 B Update
JMART    2025-01-30 2025-02-04  11.30  10.80  -4.42%  18,272,786  12.00  10.80 B Update
JMT      2025-01-30 2025-02-04  15.30  14.70  -3.92%  40,613,218  15.70  14.70 S Update
KCE      2025-01-29 2025-02-04  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-02-04' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-22,2025-02-04,5.15,5.35,"221,530,420",5.35,5.10,B,SET,0.20,3.88%
1,AIMIRT,2025-01-31,2025-02-04,10.50,10.50,"733,640",10.50,10.40,B,SET,0.00,0.00%
2,CPNREIT,2025-01-31,2025-02-04,12.10,12.20,"4,424,200",12.40,12.10,B,SET,0.10,0.83%
3,DIF,2025-01-30,2025-02-04,8.40,8.40,"71,832,660",8.55,8.35,B,SET,0.00,0.00%
4,GVREIT,2025-02-03,2025-02-04,6.75,6.60,"3,709,978",6.75,6.50,B,SET,-0.15,-2.22%
5,JMART,2025-01-30,2025-02-04,11.30,10.80,"36,545,572",12.00,10.80,B,SET50,-0.50,-4.42%
6,KCE,2025-01-29,2025-02-04,22.50,21.00,"58,047,096",23.20,21.00,B,SET100,-1.50,-6.67%
7,MCS,2025-01-29,2025-02-04,6.25,6.10,"1,076,204",6.30,6.10,B,SET,-0.15,-2.40%
8,NER,2025-01-29,2025-02-04,4.98,4.88,"55,572,172",5.05,4.88,B,SET,-0.10,-2.01%
9,SENA,2025-01-29,2025-02-04,2.18,2.16,"5,156,960",2.20,2.16,B,SET,-0.02,-0.92%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
6,KCE,2025-01-29,2025-02-04,22.50,21.00,"58,047,096",23.20,21.00,B,SET100,-1.50,-6.67%
16,RCL,2025-01-27,2025-02-04,24.90,23.50,"36,074,056",25.50,23.50,S,SET100,-1.40,-5.62%


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [67]:
conlite.close()